In [8]:
# load csv
import pandas as pd
csv_path = r"https://docs.google.com/spreadsheets/d/1EvIFLGpsKvuOgdX6F9MYlGl9rCLjbKHPlE8srBDMwYo/edit?usp=sharing"

# Download file from Google Sheets
url = csv_path.replace('/edit?usp=sharing', '/export?format=csv')
df = pd.read_csv(url)

df.columns


Index(['id', 'date', 'writer', 'model', 'screen_size', 'chip', 'core', 'ram',
       'ssd', 'apple_care_plus', 'unused', 'price', 'price for statistics',
       'platform', 'url', 'title', 'content'],
      dtype='object')

In [9]:
item_csv_path = r"https://docs.google.com/spreadsheets/d/1eRkhdeDaD60IOajReYVRId28CM4IKQJiheL-yBWfzUw/edit?usp=sharing"
url = item_csv_path.replace('/edit?usp=sharing', '/export?format=xlsx')


item_df = pd.read_excel(url, sheet_name="Mac-Docs")

api_db = {}

for index, row in item_df.iterrows():
    value = (row["item id"], row["option id"])
    key = (row["item"], row["CPU"], row["CPU Core"], row["GPU Core"], row["RAM"], row["SSD"])
    print(key)
    api_db[key] = value


('Mac Mini', 'M1', 8, 8, 8, '256GB')
('Mac Mini', 'M1', 8, 8, 8, '512GB')
('Mac Mini', 'M1', 8, 8, 8, '1TB')
('Mac Mini', 'M1', 8, 8, 16, '256GB')
('Mac Mini', 'M1', 8, 8, 16, '512GB')
('Mac Mini', 'M1', 8, 8, 16, '1TB')
('Mac Mini', 'M2', 8, 10, 8, '256GB')
('Mac Mini', 'M2', 8, 10, 8, '512GB')
('Mac Mini', 'M2', 8, 10, 8, '1TB')
('Mac Mini', 'M2', 8, 10, 16, '256GB')
('Mac Mini', 'M2', 8, 10, 16, '512GB')
('Mac Mini', 'M2', 8, 10, 16, '1TB')
('Mac Mini', 'M2Pro', 10, 16, 16, '512GB')
('Mac Mini', 'M2Pro', 10, 16, 16, '1TB')
('Mac Mini', 'M2Pro', 10, 16, 32, '512GB')
('Mac Mini', 'M2Pro', 10, 16, 32, '1TB')
('Mac Mini', 'M2Pro', 12, 19, 16, '512GB')
('Mac Mini', 'M2Pro', 12, 19, 16, '1TB')
('Mac Mini', 'M2Pro', 12, 19, 32, '512GB')
('Mac Mini', 'M2Pro', 12, 19, 32, '1TB')
('Macbook Air', 'M1', 8, 7, 8, '256GB')
('Macbook Air', 'M1', 8, 7, 8, '512GB')
('Macbook Air', 'M1', 8, 7, 8, '1TB')
('Macbook Air', 'M1', 8, 7, 16, '256GB')
('Macbook Air', 'M1', 8, 7, 16, '512GB')
('Macbook Air', 

In [19]:
# API Docs를 기반으로 CSV 생성
import datetime
import warnings
warnings.filterwarnings('ignore')

datas = []
for key in api_db:
    model, chip, cpu_core, gpu_core, ram, ssd = key
    core = ",".join([str(cpu_core), str(gpu_core)])
    ram = str(ram) + "GB"
    if "Macbook Pro" in model:
        screen_size = model.replace("Macbook Pro", "").strip()
        model = "Macbook Pro"
        entity_list = df[(df["model"] == model) & (df["screen_size"] == screen_size) & (df["chip"] == chip) & (df["core"] == core) & (df["ram"] == ram) & (df["ssd"] == ssd)]
    else:
        entity_list = df[(df["model"] == model) & (df["chip"] == chip) & (df["core"] == core) & (df["ram"] == ram) & (df["ssd"] == ssd)]
        screen_size = ""
    entity_list.loc[:, "price for statistics"] = entity_list["price for statistics"].str.replace(",", "").astype(int)
    item_id, option_id = api_db[key]
    for unopened in [True, False]:
        option_entity_list = entity_list[entity_list["unused"] == unopened]

        start_date = datetime.datetime(2023, 3, 1)
        for week in range(9):
            end_date = start_date + datetime.timedelta(days=31)
            date_entity_list = option_entity_list[(start_date.strftime("%Y-%m-%d") <= option_entity_list["date"]) & (option_entity_list["date"] <= end_date.strftime("%Y-%m-%d"))]

            if date_entity_list.size <= 1:
                datas.append({
                    "item_id": item_id,
                    "option_id": option_id,
                    "start_date": start_date.strftime("%Y-%m-%d"),
                    "end_date": end_date.strftime("%Y-%m-%d"),
                    "unopened": unopened,
                    "mean_std": round(date_entity_list["price for statistics"].mean()) if date_entity_list.size == 1 else 0,
                    "mean_20": 0,
                    "mean_80": 0,
                })
                start_date += datetime.timedelta(days=7)
                continue

            # 평균
            mean = date_entity_list["price for statistics"].mean()

            # 중앙 값
            median = date_entity_list["price for statistics"].median()

            # 표준 편차
            std = date_entity_list["price for statistics"].std()

            # 평균으로부터 표준편차 이내의 값 평균
            mean_std = date_entity_list[(date_entity_list["price for statistics"] >= (mean - std)) & (date_entity_list["price for statistics"] <= (mean + std))]["price for statistics"].mean()

            # 상위 0 ~ 20% 사이 값들의 평균
            top_20_percentile = date_entity_list["price for statistics"].quantile(0.8)
            top_20_percent_values = date_entity_list[date_entity_list["price for statistics"] >= top_20_percentile]
            mean_20 = top_20_percent_values.mean()["price for statistics"]

            # 하위 20% 평균
            bottom_20_percentile = date_entity_list["price for statistics"].quantile(0.2)
            bottom_20_percent_values = date_entity_list[date_entity_list["price for statistics"] <= bottom_20_percentile]
            mean_80 = bottom_20_percent_values.mean()["price for statistics"]

            if mean_20 == mean_80:
                mean_std = mean_20
                mean_20 = mean_80 = 0

            datas.append({
                "item_id": item_id,
                "option_id": option_id,
                "start_date": start_date.strftime("%Y-%m-%d"),
                "end_date": end_date.strftime("%Y-%m-%d"),
                "unopened": unopened,
                "mean_std": round(mean_std),
                "mean_20": round(mean_20),
                "mean_80": round(mean_80),
            })

            start_date += datetime.timedelta(days=7)

datas = pd.DataFrame(datas)
datas.to_csv("mac-data.csv", index=False)




In [21]:
import time
import requests

# API Call
api_url_template = "https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/{}/option/{}"

# read csv
df = pd.read_csv("mac-data.csv")

for index, row in df.iterrows():
    item_id = row["item_id"]
    option_id = row["option_id"]
    date = row["end_date"]
    mean = row["mean_std"]
    high = row["mean_20"]
    low = row["mean_80"]
    unopened = row["unopened"]


    api_url = api_url_template.format(item_id, option_id)
    if unopened:
        api_url += "?unopened=true"
    else:
        api_url += "?unopened=false"

    print(api_url)

    data = {
        "date": date,
        "mid": int(mean),
        "high": int(high),
        "low": int(low),
    }


    print(f"{data}")
    response = requests.post(api_url, json=data)
    print(response.status_code)
    print(response.text)
    print()
    time.sleep(2)


https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-01', 'mid': 0, 'high': 0, 'low': 0}
200
{"_writeTime":{"_seconds":1686289055,"_nanoseconds":567836000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-08', 'mid': 0, 'high': 0, 'low': 0}
200
{"_writeTime":{"_seconds":1686289058,"_nanoseconds":84571000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-15', 'mid': 0, 'high': 0, 'low': 0}
200
{"_writeTime":{"_seconds":1686289060,"_nanoseconds":660216000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-22', 'mid': 0, 'high': 0, 'low': 0}
200
{"_writeTime":{"_seconds":1686289066,"_nanoseconds":394261000}}

https://asia-northeast3-macguider-44e7e.cloudfunctions.net/api/item/1/option/1?unopened=true
{'date': '2023-04-29', 'mid': 0, 'high': 0, 